In [1]:
import os
import pandas as pd

In [2]:
root_dir = r"D:\Studying\UoR\1. Data Mining\Final_Project\data"

for file_name in os.listdir(root_dir):
    file_path = os.path.join(root_dir, file_name)
    df = pd.read_csv(file_path)
    df.drop_duplicates("url", inplace=True)
    df.to_csv(file_path, index=None)
    print(f"{file_name} has {df.shape[0]} examples")

ap_canada.csv has 60 examples
cbc.csv has 1546 examples
china_daily.csv has 8728 examples
global-news.csv has 11072 examples
moscow_times.csv has 13572 examples
reuters_canada.csv has 4233 examples
reuters_china.csv has 5666 examples
reuters_russia.csv has 5845 examples


In [3]:
from datetime import datetime
from collections import defaultdict
from pprint import pprint

In [4]:
dict = defaultdict(list)

for file_name in os.listdir(root_dir):
    file_path = os.path.join(root_dir, file_name)
    df = pd.read_csv(file_path)
    root_name = file_name.split(".")[0]
    
    df.date_publish.fillna("", inplace=True)
    time = df.date_publish.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S") if x else "")
    time = time.apply(lambda x: x.year).value_counts()
    
    print(root_name)
    print(time)

ap_canada
2023    60
Name: date_publish, dtype: int64


d:\Studying\UoR\1. Data Mining\Final_Project\venv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  if __name__ == "__main__":


cbc
2023.0    514
2021.0    339
2022.0    166
2019.0    146
2020.0     80
2015.0     22
2017.0     10
2016.0      9
2018.0      8
2014.0      4
2013.0      2
2011.0      1
2007.0      1
Name: date_publish, dtype: int64
china_daily
2017.0    2023
2018.0    1833
2019.0    1239
2021.0    1066
2020.0     921
2023.0     861
2022.0     770
2014.0       1
Name: date_publish, dtype: int64
global-news
2023.0    9928
2019.0    1140
2014.0       3
Name: date_publish, dtype: int64
moscow_times
2022    3085
2019    2778
2020    2632
2023    2570
2021    2507
Name: date_publish, dtype: int64
reuters_canada
2023.0    3332
2022.0     885
2012.0       3
2021.0       3
2015.0       2
2016.0       2
2020.0       2
2013.0       1
2014.0       1
2017.0       1
Name: date_publish, dtype: int64
reuters_china
2023    5663
2022       3
Name: date_publish, dtype: int64
reuters_russia
2023.0    5173
2022.0     670
Name: date_publish, dtype: int64


In [3]:
file_path = r"D:\Studying\UoR\1. Data Mining\Final_Project\data\global-news.csv"
df = pd.read_csv(file_path)
df.shape

(6365, 16)

In [4]:
df.date_publish.sample(10)

2136    2023-09-20 14:12:59
4324    2023-06-22 10:07:15
6165    2023-06-08 12:55:47
3037    2023-08-31 14:00:14
2606    2023-09-07 18:21:17
724     2023-09-21 12:01:59
5187    2023-06-02 12:09:22
522     2023-10-09 20:57:33
5128    2023-06-30 15:16:41
334     2023-10-13 10:00:17
Name: date_publish, dtype: object

In [5]:
df.date_publish.fillna("", inplace=True)

In [6]:
time = df.date_publish.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S") if x else "")

In [21]:
x = time.apply(lambda x: x.year).value_counts().to_dict()
# x.index
# x.values
x

{2017.0: 2023,
 2018.0: 1833,
 2019.0: 1239,
 2021.0: 1066,
 2020.0: 921,
 2023.0: 861,
 2022.0: 770,
 2014.0: 1}

In [7]:
df.sample(3)

,authors,date_download,date_modify,date_publish,description,filename,image_url,language,localpath,maintext,source_domain,text,title,title_page,title_rss,url
2743,"['Alex Cooke', 'Global News']",2023-11-18 22:56:05,NaN,2023-08-01 12:01:38,Tuesday marked the third annual Emancipation D...,https%3A%2F%2Fglobalnews.ca%2Fnews%2F9869530%2...,https://globalnews.ca/wp-content/uploads/2022/...,en,NaN,Send this page to someone via email\nTuesday m...,globalnews.ca,NaN,Emancipation Day in N.S.: A time to reflect on...,NaN,NaN,https://globalnews.ca/news/9869530/ns-emancipa...
1039,"['Sam Thompson', 'Global News']",2023-11-18 22:18:42,NaN,2023-11-10 11:37:11,Winnipeg police say a 28-year-old woman faces ...,https%3A%2F%2Fglobalnews.ca%2Fnews%2F10084459%...,https://globalnews.ca/wp-content/uploads/2023/...,en,NaN,Send this page to someone via email\nA 28-year...,globalnews.ca,NaN,Winnipeg hockey coach faces 15 charges in sexu...,NaN,NaN,https://globalnews.ca/news/10084459/winnipeg-c...
1810,"['Greg Davis', 'Global News']",2023-11-18 22:35:22,NaN,2023-09-29 10:50:50,A diesel fuel spill in September 2020 entered ...,https%3A%2F%2Fglobalnews.ca%2Fnews%2F9994362%2...,https://globalnews.ca/wp-content/uploads/2023/...,en,NaN,Send this page to someone via email\nA tractor...,globalnews.ca,NaN,"Walmart, trailer rental company fined $155K fo...",NaN,NaN,https://globalnews.ca/news/9994362/walmart-tra...
